In [362]:
# standard library imports
import os

# third party imports
import pandas as pd

# local imports


data_dir = os.path.join(os.path.dirname("__file__"), "..", "..", "data")
clean_data_dir = os.path.join(data_dir, "clean")

In [363]:
ufcstats_events = pd.read_csv(os.path.join(clean_data_dir, "UFC Stats", "events.csv"))
event_ids = ufcstats_events.loc[ufcstats_events["is_ufc_event"] == 1, "id"].values
ufcstats_bouts = pd.read_csv(os.path.join(clean_data_dir, "UFC Stats", "bouts.csv"))
ufcstats_bouts = ufcstats_bouts.loc[ufcstats_bouts["event_id"].isin(event_ids)].reset_index(drop=True)
ufcstats_bouts

,id,event_id,bout_order,red_fighter_id,blue_fighter_id,red_outcome,blue_outcome,weight_class,type_verbose,performance_bonus,outcome_method,outcome_method_details,end_round,end_round_time_seconds,round_time_format,total_time_seconds
0,567a09fd200cfa05,6420efac0578988b,1,279093302a6f44b3,96eff1a628adcc7f,W,L,Open Weight,Open Weight Bout,0,KO/TKO,Kick to Head On Ground,1,26,No Time Limit,26
1,2d2bbc86e941e05c,6420efac0578988b,2,598a58db87b890ee,d3711d3784b76255,W,L,Open Weight,Open Weight Bout,0,KO/TKO,to Corner Stoppage,1,260,No Time Limit,260
2,cecdc0da584274b9,6420efac0578988b,3,429e7d3725852ce9,a5c53b3ddb31cc7d,W,L,Open Weight,Open Weight Bout,0,Submission,Other Position - Mount,1,138,No Time Limit,138
3,46acd54cc0c905fb,6420efac0578988b,4,63b65af1c5cb02cb,46c8ec317aff28ac,W,L,Open Weight,Open Weight Bout,0,Submission,Heel Hook From Guard,1,109,No Time Limit,109
4,ac7ca2ec38b96c1a,6420efac0578988b,5,279093302a6f44b3,598a58db87b890ee,W,L,Open Weight,Open Weight Bout,0,KO/TKO,to Corner Stoppage,1,59,No Time Limit,59
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7977,5238f6470d0557fb,72c9c2eadfc3277e,9,0232cabbc30a2372,850266b3dc4e506e,L,W,Bantamweight,Bantamweight Bout,0,Decision - Split,Derek Cleary 27 - 30. Chris Lee 29 - 28. Eric ...,3,300,3 Rnd (5-5-5),900
7978,7b1bc4ff776f12c1,72c9c2eadfc3277e,10,71171fc96445bf65,e4277e87a789d687,L,W,Light Heavyweight,Light Heavyweight Bout,1,KO/TKO,Punch to Head At Distance,3,224,3 Rnd (5-5-5),824
7979,1a635a5e4551e7d5,72c9c2eadfc3277e,11,5d1b7e3dd9e11074,294aa73dbf37d281,W,L,Flyweight,Flyweight Bout,0,KO/TKO,Punches to Head At Distance,3,117,3 Rnd (5-5-5),717
7980,7521015554088962,72c9c2eadfc3277e,12,d247691a6c0e9034,aa171d55d4b5208f,W,L,Featherweight,Featherweight Bout,0,KO/TKO,Punch to Head At Distance,3,96,3 Rnd (5-5-5),696


In [364]:
# load linkage tables
events_linkage = pd.read_csv(os.path.join(clean_data_dir, "events_linkage.csv"))
fighters_linkage = pd.read_csv(os.path.join(clean_data_dir, "fighters_linkage.csv"))

## Tapology

In [365]:
tapology_bouts = pd.read_csv(os.path.join(clean_data_dir, "Tapology", "bouts.csv"))

# temp1 = tapology_bouts.iloc[345:]
# temp2 = ufcstats_bouts.iloc[345:]
# all(temp2["id"].values == temp1["ufcstats_id"].values)

# first 345 fights need to be matched, otherwise all good
temp_match1 = pd.DataFrame(
    {
        "ufcstats_id": ufcstats_bouts.loc[345:, "id"].values,
        "tapology_id": tapology_bouts.loc[345:, "id"].values,
    }
)

# deal with rest
ufcstats_bouts2 = ufcstats_bouts.loc[:344].copy()
tapology_bouts2 = tapology_bouts.loc[:344].copy()
tapology_bouts2 = tapology_bouts2[["id", "event_id", "bout_order", "fighter_1_id", "fighter_2_id"]]

tapology_bouts2_temp = tapology_bouts2.merge(events_linkage[["ufcstats_id", "tapology_id"]].rename(columns={"ufcstats_id": "ufcstats_event_id", "tapology_id": "event_id"}), how="left", on="event_id").drop(columns=["event_id"])
tapology_bouts2_temp = tapology_bouts2_temp.merge(fighters_linkage[["ufcstats_id", "tapology_id"]].rename(columns={"ufcstats_id": "ufcstats_fighter_1_id", "tapology_id": "fighter_1_id"}), how="left", on="fighter_1_id").drop(columns=["fighter_1_id"])
tapology_bouts2_temp = tapology_bouts2_temp.merge(fighters_linkage[["ufcstats_id", "tapology_id"]].rename(columns={"ufcstats_id": "ufcstats_fighter_2_id", "tapology_id": "fighter_2_id"}), how="left", on="fighter_2_id").drop(columns=["fighter_2_id"])
tapology_bouts2_temp = tapology_bouts2_temp.rename(columns={"id": "tapology_id"})
temp_match2 = ufcstats_bouts2.merge(tapology_bouts2_temp, how="inner", left_on=["event_id", "bout_order", "red_fighter_id", "blue_fighter_id"], right_on=["ufcstats_event_id", "bout_order", "ufcstats_fighter_1_id", "ufcstats_fighter_2_id"])[["id", "tapology_id"]]
temp_match2 = temp_match2.rename(columns={"id": "ufcstats_id"})

matching1 = pd.concat([temp_match2, temp_match1], ignore_index=True).reset_index(drop=True)

## ESPN

In [366]:
espn_bouts = pd.read_csv(os.path.join(clean_data_dir, "ESPN", "bouts.csv")).drop(columns=["winner_id", "card_segment"])
espn_bouts = espn_bouts.merge(events_linkage[["ufcstats_id", "espn_id"]].rename(columns={"ufcstats_id": "ufcstats_event_id", "espn_id": "event_id"}), how="left", on="event_id").drop(columns=["event_id"])
espn_bouts = espn_bouts.merge(fighters_linkage[["ufcstats_id", "espn_id"]].rename(columns={"ufcstats_id": "ufcstats_fighter_1_id", "espn_id": "fighter_1_id"}), how="left", on="fighter_1_id").drop(columns=["fighter_1_id"])
espn_bouts = espn_bouts.merge(fighters_linkage[["ufcstats_id", "espn_id"]].rename(columns={"ufcstats_id": "ufcstats_fighter_2_id", "espn_id": "fighter_2_id"}), how="left", on="fighter_2_id").drop(columns=["fighter_2_id"])
espn_bouts = espn_bouts.rename(columns={"id": "espn_id"})

temp_match1 = ufcstats_bouts.merge(espn_bouts, how="inner", left_on=["event_id", "bout_order", "red_fighter_id", "blue_fighter_id"], right_on=["ufcstats_event_id", "bout_order", "ufcstats_fighter_1_id", "ufcstats_fighter_2_id"])[["id", "espn_id"]]
temp_match1 = temp_match1.rename(columns={"id": "ufcstats_id"})

espn_bouts2 = espn_bouts.loc[~espn_bouts["espn_id"].isin(temp_match1["espn_id"].values)].copy()
ufcstats_bouts2 = ufcstats_bouts.loc[~ufcstats_bouts["id"].isin(temp_match1["ufcstats_id"].values)].copy()
temp_match2 = ufcstats_bouts2.merge(espn_bouts2, how="inner", left_on=["event_id", "bout_order", "red_fighter_id", "blue_fighter_id"], right_on=["ufcstats_event_id", "bout_order", "ufcstats_fighter_2_id", "ufcstats_fighter_1_id"])[["id", "espn_id"]]
temp_match2 = temp_match2.rename(columns={"id": "ufcstats_id"})

espn_bouts3 = espn_bouts2.loc[~espn_bouts2["espn_id"].isin(temp_match2["espn_id"].values)].copy()
ufcstats_bouts3 = ufcstats_bouts2.loc[~ufcstats_bouts2["id"].isin(temp_match2["ufcstats_id"].values)].copy()
temp_match3 = ufcstats_bouts3.merge(espn_bouts3, how="inner", left_on=["event_id", "red_fighter_id", "blue_fighter_id"], right_on=["ufcstats_event_id", "ufcstats_fighter_1_id", "ufcstats_fighter_2_id"])[["id", "espn_id"]]
temp_match3 = temp_match3.rename(columns={"id": "ufcstats_id"})

espn_bouts4 = espn_bouts3.loc[~espn_bouts3["espn_id"].isin(temp_match3["espn_id"].values)].copy()
ufcstats_bouts4 = ufcstats_bouts3.loc[~ufcstats_bouts3["id"].isin(temp_match3["ufcstats_id"].values)].copy()
temp_match4 = ufcstats_bouts4.merge(espn_bouts4, how="inner", left_on=["event_id", "red_fighter_id", "blue_fighter_id"], right_on=["ufcstats_event_id", "ufcstats_fighter_2_id", "ufcstats_fighter_1_id"])[["id", "espn_id"]]
temp_match4 = temp_match4.rename(columns={"id": "ufcstats_id"})

matching2 = pd.concat([temp_match1, temp_match2, temp_match3, temp_match4], ignore_index=True).reset_index(drop=True)
bout_ids = ufcstats_bouts["id"].values.tolist()

# sort by ufcstats_id in order to match with ufcstats_bouts
matching2 = matching2.sort_values(by="ufcstats_id", key=lambda x: x if x.name != "ufcstats_id" else x.map(lambda b: bout_ids.index(b))).reset_index(drop=True)

## FightOdds.io

In [367]:
fightoddsio_bouts = pd.read_csv(os.path.join(clean_data_dir, "FightOdds.io", "bouts.csv"))[["id", "event_id", "fighter_1_id", "fighter_2_id"]]
fightoddsio_bouts = fightoddsio_bouts.merge(events_linkage[["ufcstats_id", "fightoddsio_id"]].rename(columns={"ufcstats_id": "ufcstats_event_id", "fightoddsio_id": "event_id"}), how="left", on="event_id").drop(columns=["event_id"])
fightoddsio_bouts = fightoddsio_bouts.merge(fighters_linkage[["ufcstats_id", "fightoddsio_id"]].rename(columns={"ufcstats_id": "ufcstats_fighter_1_id", "fightoddsio_id": "fighter_1_id"}), how="left", on="fighter_1_id").drop(columns=["fighter_1_id"])
fightoddsio_bouts = fightoddsio_bouts.merge(fighters_linkage[["ufcstats_id", "fightoddsio_id"]].rename(columns={"ufcstats_id": "ufcstats_fighter_2_id", "fightoddsio_id": "fighter_2_id"}), how="left", on="fighter_2_id").drop(columns=["fighter_2_id"])
fightoddsio_bouts = fightoddsio_bouts.rename(columns={"id": "fightoddsio_id"})

# fightoddsio missing first kazushi sakuraba vs. marcus silveira fight, which ended in NC
ufcstats_bouts_temp = ufcstats_bouts.loc[ufcstats_bouts["id"] != "2750ac5854e8b28b"].copy()

temp_match1 = ufcstats_bouts_temp.merge(fightoddsio_bouts, how="inner", left_on=["event_id", "red_fighter_id", "blue_fighter_id"], right_on=["ufcstats_event_id", "ufcstats_fighter_1_id", "ufcstats_fighter_2_id"])[["id", "fightoddsio_id"]]
temp_match1 = temp_match1.rename(columns={"id": "ufcstats_id"})

fightoddsio_bouts2 = fightoddsio_bouts.loc[~fightoddsio_bouts["fightoddsio_id"].isin(temp_match1["fightoddsio_id"].values)].copy()
ufcstats_bouts2 = ufcstats_bouts_temp.loc[~ufcstats_bouts_temp["id"].isin(temp_match1["ufcstats_id"].values)].copy()
temp_match2 = ufcstats_bouts2.merge(fightoddsio_bouts2, how="inner", left_on=["event_id", "red_fighter_id", "blue_fighter_id"], right_on=["ufcstats_event_id", "ufcstats_fighter_2_id", "ufcstats_fighter_1_id"])[["id", "fightoddsio_id"]]
temp_match2 = temp_match2.rename(columns={"id": "ufcstats_id"})

matching3 = pd.concat([temp_match1, temp_match2], ignore_index=True).reset_index(drop=True)
bout_ids = ufcstats_bouts["id"].values.tolist()

# sort by ufcstats_id in order to match with ufcstats_bouts
matching3 = matching3.sort_values(by="ufcstats_id", key=lambda x: x if x.name != "ufcstats_id" else x.map(lambda b: bout_ids.index(b))).reset_index(drop=True)

## MMA Decisions

In [368]:
# i'm gonna cheat once again and use some work i did for case studies (s&ds 625)
df = pd.read_csv("mmadecisions_merged_final.csv")
temp_match = df[["ufcstats_bout_id", "mmadecisions_bout_id"]].copy().drop_duplicates().reset_index(drop=True).rename(columns={"ufcstats_bout_id": "ufcstats_id", "mmadecisions_bout_id": "mmadecisions_id"})

mmadecisions_events = pd.read_csv(os.path.join(clean_data_dir, "MMA Decisions", "events.csv"))
event_ids = mmadecisions_events.loc[mmadecisions_events["promotion"] == "UFC", "id"].values
mmadecisions_bouts = pd.read_csv(os.path.join(clean_data_dir, "MMA Decisions", "bouts.csv"))
mmadecisions_bouts = mmadecisions_bouts.loc[mmadecisions_bouts["event_id"].isin(event_ids)].reset_index(drop=True)[["id", "event_id", "fighter_1_id", "fighter_2_id"]]
mmadecisions_bouts = mmadecisions_bouts.loc[~mmadecisions_bouts["id"].isin(temp_match["mmadecisions_id"].values)].copy()
mmadecisions_bouts = mmadecisions_bouts.merge(events_linkage[["ufcstats_id", "mmadecisions_id"]].rename(columns={"ufcstats_id": "ufcstats_event_id", "mmadecisions_id": "event_id"}), how="left", on="event_id").drop(columns=["event_id"])
mmadecisions_bouts = mmadecisions_bouts.merge(fighters_linkage[["ufcstats_id", "mmadecisions_id"]].rename(columns={"ufcstats_id": "ufcstats_fighter_1_id", "mmadecisions_id": "fighter_1_id"}), how="left", on="fighter_1_id").drop(columns=["fighter_1_id"])
mmadecisions_bouts = mmadecisions_bouts.merge(fighters_linkage[["ufcstats_id", "mmadecisions_id"]].rename(columns={"ufcstats_id": "ufcstats_fighter_2_id", "mmadecisions_id": "fighter_2_id"}), how="left", on="fighter_2_id").drop(columns=["fighter_2_id"])
mmadecisions_bouts = mmadecisions_bouts.rename(columns={"id": "mmadecisions_id"})

ufcstats_bouts_temp = ufcstats_bouts.loc[ufcstats_bouts["event_id"].isin(mmadecisions_bouts["ufcstats_event_id"])].copy()
ufcstats_bouts_temp = ufcstats_bouts_temp.loc[~ufcstats_bouts_temp["id"].isin(temp_match["ufcstats_id"].values)].copy()
ufcstats_bouts_temp = ufcstats_bouts_temp.loc[(ufcstats_bouts_temp["outcome_method"].str.contains("Decision")) | (ufcstats_bouts_temp["outcome_method"] == "Overturned") | (ufcstats_bouts_temp["outcome_method"] == "Other")].copy()


temp_match1 = ufcstats_bouts_temp.merge(mmadecisions_bouts, how="inner", left_on=["event_id", "red_fighter_id", "blue_fighter_id"], right_on=["ufcstats_event_id", "ufcstats_fighter_1_id", "ufcstats_fighter_2_id"])[["id", "mmadecisions_id"]]
temp_match1 = temp_match1.rename(columns={"id": "ufcstats_id"})

mmadecisions_bouts2 = mmadecisions_bouts.loc[~mmadecisions_bouts["mmadecisions_id"].isin(temp_match1["mmadecisions_id"].values)].copy()
ufcstats_bouts2 = ufcstats_bouts_temp.loc[~ufcstats_bouts_temp["id"].isin(temp_match1["ufcstats_id"].values)].copy()
temp_match2 = ufcstats_bouts2.merge(mmadecisions_bouts2, how="inner", left_on=["event_id", "red_fighter_id", "blue_fighter_id"], right_on=["ufcstats_event_id", "ufcstats_fighter_2_id", "ufcstats_fighter_1_id"])[["id", "mmadecisions_id"]]
temp_match2 = temp_match2.rename(columns={"id": "ufcstats_id"})

matching4 = pd.concat([temp_match, temp_match1, temp_match2], ignore_index=True).reset_index(drop=True)
bout_ids = ufcstats_bouts["id"].values.tolist()

# sort by ufcstats_id in order to match with ufcstats_bouts
matching4 = matching4.sort_values(by="ufcstats_id", key=lambda x: x if x.name != "ufcstats_id" else x.map(lambda b: bout_ids.index(b))).reset_index(drop=True)

## Bet MMA

In [375]:
betmma_events = pd.read_csv(os.path.join(clean_data_dir, "Bet MMA", "events.csv"))
event_ids = betmma_events.loc[betmma_events["is_ufc_event"] == 1, "id"].values
betmma_bouts = pd.read_csv(os.path.join(clean_data_dir, "Bet MMA", "bouts.csv"))
betmma_bouts = betmma_bouts.loc[betmma_bouts["event_id"].isin(event_ids)].reset_index(drop=True)
betmma_bouts = betmma_bouts.merge(events_linkage[["ufcstats_id", "betmma_id"]].rename(columns={"ufcstats_id": "ufcstats_event_id", "betmma_id": "event_id"}), how="left", on="event_id").drop(columns=["event_id"])
betmma_bouts = betmma_bouts.merge(fighters_linkage[["ufcstats_id", "betmma_id"]].rename(columns={"ufcstats_id": "ufcstats_fighter_1_id", "betmma_id": "fighter_1_id"}), how="left", on="fighter_1_id").drop(columns=["fighter_1_id"])
betmma_bouts = betmma_bouts.merge(fighters_linkage[["ufcstats_id", "betmma_id"]].rename(columns={"ufcstats_id": "ufcstats_fighter_2_id", "betmma_id": "fighter_2_id"}), how="left", on="fighter_2_id").drop(columns=["fighter_2_id"])
betmma_bouts = betmma_bouts.rename(columns={"id": "betmma_id"})

ufcstats_bouts_temp = ufcstats_bouts.loc[ufcstats_bouts["event_id"].isin(betmma_bouts["ufcstats_event_id"])].copy()
temp_match1 = ufcstats_bouts_temp.merge(betmma_bouts, how="inner", left_on=["event_id", "red_fighter_id", "blue_fighter_id"], right_on=["ufcstats_event_id", "ufcstats_fighter_1_id", "ufcstats_fighter_2_id"])[["id", "betmma_id"]]
temp_match1 = temp_match1.rename(columns={"id": "ufcstats_id"})

betmma_bouts2 = betmma_bouts.loc[~betmma_bouts["betmma_id"].isin(temp_match1["betmma_id"].values)].copy()
ufcstats_bouts2 = ufcstats_bouts_temp.loc[~ufcstats_bouts_temp["id"].isin(temp_match1["ufcstats_id"].values)].copy()
temp_match2 = ufcstats_bouts2.merge(betmma_bouts2, how="inner", left_on=["event_id", "red_fighter_id", "blue_fighter_id"], right_on=["ufcstats_event_id", "ufcstats_fighter_2_id", "ufcstats_fighter_1_id"])[["id", "betmma_id"]]
temp_match2 = temp_match2.rename(columns={"id": "ufcstats_id"})

matching5 = pd.concat([temp_match1, temp_match2], ignore_index=True).reset_index(drop=True)
bout_ids = ufcstats_bouts["id"].values.tolist()

# sort by ufcstats_id in order to match with ufcstats_bouts
matching5 = matching5.sort_values(by="ufcstats_id", key=lambda x: x if x.name != "ufcstats_id" else x.map(lambda b: bout_ids.index(b))).reset_index(drop=True)

In [381]:
matching = matching1.merge(matching2, on="ufcstats_id", how="left")
matching = matching.merge(matching3, on="ufcstats_id", how="left")
matching = matching.merge(matching4, on="ufcstats_id", how="left")
matching = matching.merge(matching5, on="ufcstats_id", how="left")
matching["mmadecisions_id"] = matching["mmadecisions_id"].astype("Int64")
matching["betmma_id"] = matching["betmma_id"].astype("Int64")
matching = matching[
    [
        "ufcstats_id",
        "betmma_id",
        "espn_id",
        "fightoddsio_id",
        "mmadecisions_id",
        "tapology_id",
    ]
]
matching.to_csv(os.path.join(clean_data_dir, "bouts_linkage.csv"), index=False)